# convert dump to csv

In [ ]:
import re

In [ ]:
def dump_to_csv(dump_filename):
    begin_regex = re.compile('^COPY.* stdin;$')
    header_regex = re.compile(', ')
    line_regex = re.compile('\t+')
    end_regex = re.compile('^\\\.$')
    line_count = 0
    fast_forward = True
    tables = []
    with open(dump_filename, 'r') as f:
        for line in f:
            # check if line is the COPY statement (similar to INSERT)
            if re.match(begin_regex, line):
                target_table = line[5:line.find('(')-1]
                tables.append(target_table)
                result_file = open(target_table + '_.csv', 'w')
                # CSV header, i.e. extract the column names
                header = line[line.find('(')+1:line.find(')')]
                header = re.sub(header_regex, ';', header)
                result_file.write(header)
                result_file.write('\n')
                result_file.close()
                result_file = open(target_table + '.csv', 'a')
                fast_forward = False
                continue
            # begin not yet found
            if fast_forward:
                continue
            # end of COPY reached
            if re.match(end_regex, line):
                line_count = 0
                fast_forward = True
                continue
            # convert tabs to ';' for CSV format
            newline = re.sub(line_regex, ';', line)
            result_file.write(newline)
    return tables

In [ ]:
tables = dump_to_csv('public_postgresql/databases/PostgreSQL.sql')

# evaluate gems

In [ ]:
import pandas as pd

In [ ]:
gems = pd.read_csv('rubygems.csv', ';', low_memory=False)

In [ ]:
gems.drop('name', axis=1, inplace=True)
gems.drop('updated_at', axis=1, inplace=True)
gems.drop('downloads', axis=1, inplace=True)
gems.drop('slug', axis=1, inplace=True)
gems['created_at'] = pd.to_datetime(gems['created_at'])
gems = gems.set_index('created_at')
gems['day'] = gems.index.date
#gems.set_index('day', inplace=True)
gems.rename(columns=lambda x: x.replace('id', 'gems'), inplace=True)

In [ ]:
# the actual magic :)
gems_cumsum = gems.groupby('day').count().cumsum()

In [ ]:
gems_cumsum.to_csv('rubygems_cumsum.csv', ';')

In [ ]:
import matplotlib

In [ ]:
gems_cumsum = pd.read_csv('rubygems_cumsum.csv', ';')
# plot to png
gems_cumsum.plot().get_figure().savefig('rubygems.png')

# dependencies

In [ ]:
deps = pd.read_csv('dependencies.csv', ';', low_memory=False)

In [ ]:
# drop what we don't need
deps.drop('updated_at', axis=1, inplace=True)
deps.drop('requirements', axis=1, inplace=True)
deps.drop('rubygem_id', axis=1, inplace=True)
deps.drop('version_id', axis=1, inplace=True)
deps.drop('scope', axis=1, inplace=True)
deps.drop('unresolved_name', axis=1, inplace=True)

In [ ]:
# treat created_at as datetime and use it as index since we deal with time series data
deps['created_at'] = pd.to_datetime(deps['created_at'])
deps.set_index('created_at', inplace=True)
# reduce to day only
deps['day'] = deps.index.date
deps.rename(columns=lambda x: x.replace('id', 'deps'), inplace=True)

In [ ]:
# calculate the cumulative sums of the count of newly added dependencies for each day
deps_cumsum = deps.groupby('day').count().cumsum()

In [ ]:
deps_cumsum.to_csv('dependencies_cumsum.csv', ';')

In [ ]:
#deps_cumsum = pd.read_csv('dependencies_cumsum.csv', ';')
deps_cumsum.plot().get_figure().savefig('dependencies.png')

# merge dataframes

In [1]:
import pandas as pd
deps_cumsum = pd.read_csv('dependencies_cumsum.csv', ';')
gems_cumsum = pd.read_csv('rubygems_cumsum.csv', ';')
cumsum = pd.merge(deps_cumsum, gems_cumsum, on='day')

In [2]:
cumsum['deps_per_gem'] = cumsum['deps']/cumsum['gems']

In [3]:
cumsum['day'] = pd.to_datetime(cumsum['day'])
cumsum.set_index('day', inplace=True)

In [5]:
cumsum.gems.plot(legend=True, style='b-')
cumsum.deps.plot(legend=True, style='g-')
plot = cumsum.deps_per_gem.plot(legend=True, style='r-', secondary_y=True)
fig = plot.get_figure()
fig.savefig('plot.png')
# cumsum.plot().get_figure().savefig('plot.png')